# ห้ามลบ

- ในหน้านี้พลเป็นคนทำ
- ควรแก้ ในการยิง API ผมได้ใส่ parametor link ให้คุณเอา link สับย่อยมาใส่

##สิ่งที่แก้ไข

1.ทำการลดการ import

2.ลบหรือยุบตัวแปรที่ไม่ได้ใช้ออกเพื่อให้เข้าใจง่าย

3.เปลี่ยนแปลง API  ให้สามารถส่งลิ้งก์มาด้วย

4.ได้ทำการเปลี่ยน nan เป็น "" หรือ 0 สำเร็จแล้ว

5.uri_API  เพิ่มเส้น APi ไว้ด้านบนใต้ import เพื่อให้ง่ายในการแก้ไข

In [2]:
import requests,pandas as pd
uri_API = ''
data_link_for_lazada  = {
    0: 'อุปกรณ์-อิเล็กทรอนิกส์',
    1: 'อุปกรณ์เสริม-อิเล็กทรอนิกส์', 
    2: 'ทีวีและเครื่องใช้ในบ้าน', 
    3: 'สุขภาพและความงาม', 
    4: 'ทารกและของเล่น', 
    5: 'ของชำและสัตว์เลี้ยง', 
    6: 'บ้านและไลฟ์สไตล์', 
    7: 'แฟชั่นและเครื่องประดับผู้หญิง', 
    8: 'แฟชั่นและเครื่องประดับผู้ชาย',
    9: 'กีฬาและการเดินทาง',
    10: 'ยานยนต์และรถจักรยานยนต์'
    }
# head_excel
header = ['_95X4G href', 'jBwCF src', 'jBwCF src 2','RfADt', 'ooOxS','_1cEkb', 'qzqFw', 'oa6ri']
header_Values = {
    '_95X4G href':"product",
    'jBwCF src':"image_product_1",
    'jBwCF src 2':"image_product_2",
    'IcOsH':"discount",
    'RfADt':"data_product", 
    'ooOxS':"price_product",
    '_1cEkb':"sold",
    'qzqFw':"count_review", 
    'oa6ri':"place"
}

In [3]:
def postAPI_DB(data,id_shop,title_group,i1,link):
    """
    data: text ที่ทำการ += ในตัวแปร success_data_text
    id_shop : shop1_1_1
    title_group:หมวดหมู่กลุ่ม
    i1:กลุ่มหลัก 1 
    link: link หมวดหลัก
    """
    try:
        response = requests.post(
            f"{uri_API}/addb?id={id_shop}&&web=lazada&&group={i1}&&title_group={title_group}&&link={link}",
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return {"status":200,"message":"POST API SUCCESS."}
    except:
        return {"status":404,"message":"POST API ERROR."}

In [10]:
from data_address import address as ad

def data_process(path_file,i1,i2,i3,group,link):
    """
    Process : เพื่อจัดตำแหน่งข้อมูลให้สามารถป้อนเข้า Data Base ได้
    Args:
        path_file: ไฟล์ข้อมูล excel ที่โหลดมาด้วยโปเกบอล อ่านเพื่อแปลงข้อมูลเป็น json
        i1: รอบการทำงานใหญ่ที่สุด ก็คือ หมวดหมู่
        i2: รอบการทำงานกลาง ก็คือ หมวดหมู่ สับย่อยลงมา
        i3: รอบการทำงานเล็กสุด ก็คือ หน้าแต่ละหน้าของสับย่อย ที่เราให้ไปอ่านแล้วกดโหลด
        group: _description_
    """
    try:
        read_excel = pd.read_excel(path_file);
        num_rows, num_columns = read_excel.shape
        success_data_text = ""
        for i in range(num_rows):
            data_process = {
                "product":[],
                "price_product_2":[],
                "price_product_1":[],
                "image_product_1":[],
                "discount":[],
                "image_product_2":[],
                "data_product":[],
                "price_before":[],
                "Emoji":[],
                "sold":[],
                "place":[],
                "Recommended_shops":[],
                "count_review":[],
                "maket":[],
                "group":[]
            }
            data = "Product_"+str(i+1);
            for j in range(len(header)):
                data_input = str(read_excel[header[j]][i]);
                data_process[header_Values[header[j]]]=data_input;
            # ****************************************************************
            Product = {}
            Product[data]= data_process
            Product[data]["maket"]="lazada"
            Product[data]["group"]=group
            id_shop = f'shop{i1}_{i2}_{i3}'
            # ****************************************************************
            product = Product[data]["product"]
            image_product_1 = Product[data]["image_product_1"]
            image_product_2 = Product[data]["image_product_2"]
            discount = Product[data]["discount"]
            data_product = Product[data]["data_product"]
            price_product = float(Product[data]["price_product"].replace("฿","").replace(",",""))
            price_product = (price_product<=0)and "0" or price_product
            sold = (Product[data]["sold"].split(" ")[0]=="nan")and "0" or type(Product[data]["sold"].split(" ")[0] )
            address = (Product[data]["place"]=='nan')and "" or ad[Product[data]["place"]]
            count_review = (Product[data]["count_review"]=="nan")and "0" or Product[data]["count_review"]
            maket = Product[data]["maket"]
            # ****************************************************************
            success_data_text += f'APRODUCT:::maket:::{maket}, group:::{group}, product:::{product}, price_product_2:::{""}, price_product_1:::{price_product}, image_product_1:::{image_product_1}, discount:::{discount}, image_product_2:::{image_product_2}, data_product:::{data_product}, price_before:::{""}, Emoji:::{""}, sold:::{sold}, place:::{address}, Recommended_shops:::{""}, count_review:::{count_review}'
            # ถ้าข้อมูลครบ 60 ค่อยบันทึก .json และส่ง API
            if(i==num_rows-1):
                print(success_data_text)
                print(postAPI_DB(success_data_text,id_shop,group,i1,link));
    except:
        print(f"data_process : error ==> i1:{i1}, i2:{i2}, i3:{i3}")
data_process('../Bot_shopee/Data_shopee/data_1_1_1.xlsx',1,1,1,'อุปกรณ์-อิเล็กทรอนิกส์',"")

data_process : error ==> i1:1, i2:1, i3:1


In [11]:


import requests,pandas as pd
from data_address import address as ad
uri_API = 'https://b18f-223-206-131-122.ngrok-free.app/';
header = ['col-xs-2-4 href', 'Fd4QmV src', 'FTxtVW',
       'customized-overlay-image src', 'DgXDzJ', 'bPcAVl', 'k9JZlv',
       'bx++ig 2', 'k9JZlv 2', 'OwmBnn', 'JVW3E2', 'hxLzax']
header_Values = {
    'col-xs-2-4 href':"product",
    'Fd4QmV src':"image_product_1",
    'FTxtVW':"discount",
    'customized-overlay-image src':"image_product_2",
    'DgXDzJ':"data_product", 
    'bPcAVl':"price_before",
    'k9JZlv':"price_product_1",
    'bx++ig 2':"Emoji",
    'k9JZlv 2':"price_product_2",
    'OwmBnn':"sold",
    'JVW3E2':"place", 
    'hxLzax':"Recommended_shops"
}
def check_data(path_file):
    try:
        header = ['col-xs-2-4 href', 'Fd4QmV src',
       'customized-overlay-image src', 'DgXDzJ', 'bPcAVl', 'k9JZlv',
       ]
       
        df = pd.read_excel(path_file)
        is_subset = all(item in df.columns for item in header);
        print("Check_data : ",is_subset);
        return is_subset; 

    except Exception as e:
        
        print("Check_data : ",e);  
# Read Excell
def postAPI_DB(data,id_shop,title_group,i1,link):
    """
    data: text ที่ทำการ += ในตัวแปร success_data_text
    id_shop : shop1_1_1
    title_group:หมวดหมู่กลุ่ม
    i1:กลุ่มหลัก 1 
    link: link หมวดหลัก
    """
    try:
        response = requests.post(
            f"{uri_API}addb?id={id_shop}&&web=shopee&&group={title_group}&&title_group={title_group}&&link={link}",
            headers={
                "Content-type":"application/x-www-form-urlencoded"
            },
            data={
                "data":data
            }
        )
        return {"status":200,"message":"POST API SUCCESS."}
    except:
        return {"status":404,"message":"POST API ERROR."}
def data_process(path_file,i1,i2,i3,group,link):
    try:
        find = pd.read_excel(path_file);
        data_all=[];
        df = pd.read_excel(path_file)
        num_rows, num_columns = df.shape
        Data_everthing=[];
        success_data_text = ""
        for i in range(num_rows):
            data_process = {
                "product":[],
                "price_product_2":[],
                "price_product_1":[],
                "image_product_1":[],
                "discount":[],
                "image_product_2":[],
                "data_product":[],
                "price_before":[],
                "Emoji":[],
                "sold":[],
                "place":[],
                "Recommended_shops":[],
                "count_review":[],
                "maket":[],
                "group":[]
            }
            data = "Product_"+str(i+1);
            Product = {
                data:{
                }
            }
            # เข้าถึงข้อมูลแต่ละชิ้น
            for j in range(12):
                data_input = str(find[header[j]][i]);
                # print(data_input)
                data_process[header_Values[header[j]]]=data_input;
            Product[data]=data_process;
            Product[data]["maket"] = "shopee";
            Product[data]["group"] = group;
            price_product1 = float(Product[data]["price_product_1"].replace(",",""));
            price_product2 = float(Product[data]["price_product_2"].replace(",",""));
            discount = (Product[data]["discount"]=='nan')and "" or Product[data]["discount"];
            price_product1 = (price_product1<=0)and "0" or price_product1;
            price_product2 = (price_product2<=0)and "0" or price_product2;
            address = (Product[data]["place"]=='nan')and "" or Product[data]["place"];
            sold = (Product[data]["sold"]=='nan')and "" or Product[data]["sold"];
            price_before = (Product[data]["price_before"]=='nan')and "" or Product[data]["price_before"]
            # ***************************ไอดีสินค้าหลัก*************************************
            id_shop = "shop"+str(i1)+"_"+str(i2)+"_"+str(i3);
            # ****************************************************************
            success_data_text += f'APRODUCT:::maket:::{Product[data]["maket"]}, group:::{group}, product:::{Product[data]["product"]}, price_product_2:::{price_product2}, price_product_1:::{price_product1}, image_product_1:::{Product[data]["image_product_1"]}, discount:::{discount}, image_product_2:::{Product[data]["image_product_2"]}, data_product:::{Product[data]["data_product"]}, price_before:::{price_before}, Emoji:::{Product[data]["Emoji"]}, sold:::{sold}, place:::{address}, Recommended_shops:::{Product[data]["Recommended_shops"]}'
            data_all.append(Product);
            success_data = {
                 "id":id_shop,
                 "data":data_all
            }
            # ถ้าข้อมูลครบ 60 ค่อยบันทึก .json และส่ง API
            if(i==num_rows-1):
                print(success_data_text); #ข้อมูลที่จะส่งไป API
                print(postAPI_DB(success_data_text,id_shop,group,i1,link));
        print("data_process : True")
    except Exception as e:
        print("data_process : False",e)
data_process('../Bot_shopee/Data_shopee/data_1_1_1.xlsx',1,1,1,'อุปกรณ์-อิเล็กทรอนิกส์',"")

APRODUCT:::maket:::shopee, group:::อุปกรณ์-อิเล็กทรอนิกส์, product:::https://shopee.co.th/%F0%9F%94%A5HOT%E0%B9%84%E0%B8%81%E0%B9%88%E0%B8%95%E0%B8%B1%E0%B8%99-%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%B1%E0%B8%99%E0%B8%95%E0%B8%B5-(6%E0%B8%AB%E0%B8%A1%E0%B8%B1%E0%B8%94-GOD-%E0%B8%AA%E0%B8%B8%E0%B9%88%E0%B8%A1%E0%B8%9C%E0%B8%A5%E0%B8%95%E0%B8%B7%E0%B9%88%E0%B8%99%E0%B9%80%E0%B8%97%E0%B8%9E%E0%B9%86%E0%B8%9E%E0%B8%A3%E0%B9%89%E0%B8%AD%E0%B8%A1PVP)-%E0%B8%AA%E0%B8%B8%E0%B9%88%E0%B8%A1%E0%B8%AD%E0%B8%B7%E0%B9%88%E0%B8%99%E0%B8%AD%E0%B8%B5%E0%B8%81%E0%B8%A1%E0%B8%B2%E0%B8%81%E0%B8%A1%E0%B8%B2%E0%B8%A2-%E0%B9%84%E0%B8%81%E0%B9%88%E0%B8%95%E0%B8%B1%E0%B8%99-%E0%B9%82%E0%B8%A3%E0%B8%9A%E0%B8%AD%E0%B8%81-%E0%B8%9A%E0%B8%A5%E0%B9%87%E0%B8%AD%E0%B8%81%E0%B8%9F%E0%B8%B8%E0%B8%95-blox-fruit-roblox-i.1020500864.23171691914?sp_atk=d402a59d-2a93-4eff-a9f5-9ac09e455053&xptdk=d402a59d-2a93-4eff-a9f5-9ac09e455053, price_product_2:::nan, price_product_1:::49.0, image_product_1:::https://down-th.img.susercontent.